In [1]:
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.aqua.components.optimizers.cobyla import COBYLA
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.chemistry.core import Hamiltonian, TransformationType, QubitMappingType
from qiskit.aqua.algorithms.adaptive.vqe.vqe import VQE
from qiskit.chemistry import FermionicOperator
from qiskit import Aer, IBMQ
from qiskit.chemistry.components.initial_states import HartreeFock
from qiskit.aqua.operators import WeightedPauliOperator
from qiskit import execute
from qiskit.aqua import QuantumInstance
from qiskit.chemistry.algorithms.q_equation_of_motion.q_equation_of_motion import QEquationOfMotion 
from qiskit.providers.aer import noise
from qiskit.aqua.algorithms import ExactEigensolver
from qiskit.quantum_info import Pauli
import numpy as np
import os
import logging
import copy

In [2]:
qasm_simulator = Aer.get_backend('qasm_simulator')
sv_simulator = Aer.get_backend('statevector_simulator')

In [3]:
def spin_operator(num_qubits):
    pauli_list = []
    for i in range(num_qubits):
        if i < num_qubits:
            c = 1/2
        else:
            c = -1/2
        a_z = np.asarray([0] * i + [1] + [0] * (num_qubits - i - 1), dtype=np.bool)
        a_x = np.asarray([0] * i + [0] + [0] * (num_qubits - i - 1), dtype=np.bool)
        pauli_list.append([c, Pauli(a_z, a_x)])
    op = WeightedPauliOperator(pauli_list)
    return op

In [4]:
def number_operator(num_qubits):
    h1 = np.identity(num_qubits)
    op = FermionicOperator(h1)
    num_op = op.mapping('jordan_wigner')
    return num_op

In [5]:
def exact_eigenstates(hamiltonian, num_particles, num_spin):
    num_qubits = hamiltonian.num_qubits
    exact_eigensolver = ExactEigensolver(hamiltonian, k=1<<num_qubits)
    exact_results = exact_eigensolver.run()
    
    results = [[],[]]
    
    number_op = number_operator(num_qubits)
    spin_op = spin_operator(num_qubits)
    
    for i in range(len(exact_results['eigvals'])):
        particle = round(number_op.evaluate_with_statevector(exact_results['eigvecs'][i])[0],1)
        spin = round(spin_op.evaluate_with_statevector(exact_results['eigvecs'][i])[0],1)
        if particle != num_particles or spin != num_spin:
            continue
        results[0].append(exact_results['eigvals'][i])
        results[1].append(exact_results['eigvecs'][i])
    return results

## In this example we will compute the excitation energies of a hydrogen molecule. 

Let's start by running a (classical) Hartree-Fock calculation to obtain the molecular orbitals of H$_2$.

In [6]:
two_qubit_reduction = False
qubit_mapping = 'jordan_wigner'
distance = 0.75

pyscf_driver = PySCFDriver(atom='H .0 .0 {}; H .0 .0 .0'.format(distance),
                               unit=UnitsType.ANGSTROM, charge=0, spin=0, basis='sto3g')
molecule = pyscf_driver.run()

core = Hamiltonian(transformation=TransformationType.FULL,
                   qubit_mapping=QubitMappingType.JORDAN_WIGNER,
                   two_qubit_reduction=two_qubit_reduction,
                   freeze_core=False,
                   orbital_reduction=[])
algo_input = core.run(molecule)
hamiltonian = algo_input[0]

num_orbitals = core.molecule_info['num_orbitals']
num_particles = core.molecule_info['num_particles']


### 1. Ground state calculation

The first step of the qEOM procedure is to find an approximation of the ground state wavefunction of the previously computed Hamiltonian.

In [7]:
init_state = HartreeFock(hamiltonian.num_qubits, num_orbitals, num_particles,
                             qubit_mapping=qubit_mapping, two_qubit_reduction=two_qubit_reduction)

depth = 1
var_form = UCCSD(hamiltonian.num_qubits, depth, num_orbitals, num_particles, initial_state=init_state,
                 qubit_mapping=qubit_mapping, two_qubit_reduction=two_qubit_reduction)

optimizer = COBYLA(maxiter = 5000)

algo = VQE(hamiltonian, var_form, optimizer)

results = algo.run(sv_simulator)
print(results)
energy = results['energy']
opt_params = results['opt_params']
ground_state = results['min_vector']

{'num_optimizer_evals': 50, 'min_val': -1.8426866578804222, 'opt_params': array([-5.42140099e-05,  1.01252286e-04, -1.14932471e-01]), 'eval_time': 0.628587007522583, 'eval_count': 50, 'energy': -1.8426866578804222, 'eigvals': array([-1.84268666]), 'min_vector': array([ 3.81633296e-17+2.42855418e-17j, -3.72895392e-16+1.55142555e-16j,
       -1.12995821e-16+7.63278329e-17j,  4.16333634e-17+1.38777878e-17j,
       -2.77555169e-16+5.86836874e-22j,  7.02441661e-01+7.02441661e-01j,
       -3.83350938e-05-3.83350938e-05j, -3.46944695e-16+3.46944695e-16j,
       -5.69755012e-18+9.14469645e-17j,  7.15961776e-05+7.15961776e-05j,
       -8.10907301e-02-8.10907301e-02j,  1.38777878e-17+6.93889390e-17j,
        4.38614666e-17-3.24663664e-17j, -3.76079054e-16+6.66133815e-16j,
        1.93813650e-16+6.62379765e-17j, -6.93889390e-17+4.16333634e-17j]), 'eigvecs': array([[ 3.81633296e-17+2.42855418e-17j, -3.72895392e-16+1.55142555e-16j,
        -1.12995821e-16+7.63278329e-17j,  4.16333634e-17+1.38777878

### 2. Excited states calculation

Now that we know how to prepare the ground state in the quantum computer we can measure the EOM matrix elements, reconstruct the pseudo-eigenvalue problem (below) and solve it classically. All these steps are taken care of by the subroutine below which returns the excitation energies as well as the EOM matrices. 

$
\begin{pmatrix}
    \text{M} & \text{Q}\\ 
    \text{Q*} & \text{M*}
\end{pmatrix}
\begin{pmatrix}
    \text{X}_n\\ 
    \text{Y}_n
\end{pmatrix}
= E_{0n}
\begin{pmatrix}
    \text{V} & \text{W}\\ 
    -\text{W*} & -\text{V*}
\end{pmatrix}
\begin{pmatrix}
    \text{X}_n\\ 
    \text{Y}_n
\end{pmatrix}
$

In [8]:
eom = QEquationOfMotion(hamiltonian, num_orbitals, num_particles, qubit_mapping=qubit_mapping,
                                    two_qubit_reduction=two_qubit_reduction)

excitation_energies, eom_matrices = eom.calculate_excited_states(ground_state)

print(excitation_energies)

[0.59437208 0.95791516 1.59695871]


### 3. Results

We compare those results to the exact values obtained by diagonalization of the Hamiltonian. In the previous example we computed the excitation energies of state that preserve the number of particle and spin (this is the default in the implementation of qEOM at the moment). Hence, we need to compare to the corresponding exact values. The degeneracies are alse ignored. 

In [9]:
qeom_energies = [energy]
for gap_i in excitation_energies:
    qeom_energies.append(energy+gap_i)

reference = exact_eigenstates(hamiltonian, 2, 0) #returns only the states with 2 electrons and singlet spin state.
exact_energies = []

tmp = 1000
for i in range(len(reference[0])):
    if np.abs(reference[0][i]-tmp)>1e-5:
        exact_energies.append(np.real(reference[0][i]))
        tmp = reference[0][i]

for i in range(4):
    print('State {} -> exact energy={} , qeom energy={}'.format(i, exact_energies[i], qeom_energies[i]))


State 0 -> exact energy=-1.8426866819057308 , qeom energy=-1.8426866578804222
State 1 -> exact energy=-1.248351713417759 , qeom energy=-1.2483145793851622
State 2 -> exact energy=-0.8848086402631494 , qeom energy=-0.8847714965126777
State 3 -> exact energy=-0.24576509272507635 , qeom energy=-0.24572795011853898


In [10]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-internal',group='dev-qiskit')
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q-internal', group='dev-qiskit', project='pulse-testing')>,
 <IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q-internal', group='dev-qiskit', project='pulse-testing')>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ(hub='ibm-q-internal', group='dev-qiskit', project='pulse-testing')>,
 <IBMQBackend('ibmq_poughkeepsie') from IBMQ(hub='ibm-q-internal', group='dev-qiskit', project='pulse-testing')>,
 <IBMQBackend('ibmq_boeblingen') from IBMQ(hub='ibm-q-internal', group='dev-qiskit', project='pulse-testing')>,
 <IBMQBackend('ibmq_vigo') from IBMQ(hub='ibm-q-internal', group='dev-qiskit', project='pulse-testing')>,
 <IBMQBackend('ibmq_ourense') from IBMQ(hub='ibm-q-internal', group='dev-qiskit', project='pulse-testing')>,
 <IBMQBackend('ibmq_valencia') from IBMQ(hub='ibm-q-internal', group='dev-qiskit', project='pulse-testing')>,
 <IBMQBackend('ibmq_london') from IBMQ(hub='ibm-q-internal', group='dev-qiskit', project='pulse-testing')>,
 <IBMQ

In [11]:
device = provider.get_backend('ibmq_boeblingen')
properties = device.properties()
coupling_map = device.configuration().coupling_map
noise_model = noise.device.basic_device_noise_model(properties)
basis_gates = noise_model.basis_gates
shots = 10000

quantum_instance = QuantumInstance(qasm_simulator, shots=shots, basis_gates=basis_gates, 
                                   coupling_map=coupling_map, noise_model=noise_model)

wave_function = var_form.construct_circuit(opt_params)


excitation_energies, eom_matrices = eom.calculate_excited_states(wave_function, quantum_instance = quantum_instance)

qeom_energies_noisy = [energy]
for gap_i in excitation_energies:
    qeom_energies_noisy.append(energy+gap_i)

for i in range(4):
    print('State {} -> exact energy={} , qeom energy={}'.format(i, exact_energies[i], qeom_energies_noisy[i]))

State 0 -> exact energy=-1.8426866819057308 , qeom energy=-1.8426866578804222
State 1 -> exact energy=-1.248351713417759 , qeom energy=-1.2369153580634844
State 2 -> exact energy=-0.8848086402631494 , qeom energy=-0.8975833748016108
State 3 -> exact energy=-0.24576509272507635 , qeom energy=-0.25304924746753255


In [12]:
import qiskit.tools.jupyter
%qiskit_version_table
%qiskit_copyright